In [1]:
import tensorflow as tf
import numpy as np

In [2]:
EPOCHS=10

In [6]:
class ResidualUnit(tf.keras.Model):
  def __init__(self, filter_in, filter_out, kernel_size):
    super(ResidualUnit, self).__init__()
    
    self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
    self.bn1 = tf.keras.layers.BatchNormalization()
    
    self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
    self.bn2 = tf.keras.layers.BatchNormalization()
    
    if filter_in == filter_out:
        self.identity = lambda x: x
    else:
        self.identity = tf.keras.layers.Conv2D(filter_out, (1,1), padding='same')

  def call(self, x, training=False, mask=None):
    h = self.conv1(x)
    h = self.bn1(h, training=training)
    h = tf.nn.relu(h)
        
    h = self.conv2(h)
    h = self.bn2(h, training=training)

    h=tf.keras.layers.Add()([h,self.identity(x)])

    return tf.nn.relu(h)

In [7]:
class ResnetLayer(tf.keras.Model):
  def __init__(self, filter_in, filters, kernel_size):
    super(ResnetLayer, self).__init__()
    self.sequence = list()
    for f_in, f_out in zip([filter_in] + list(filters), filters):
      self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))

  def call(self, x, training=False, mask=None):
    for unit in self.sequence:
        x = unit(x, training=training)
        
    return x

In [8]:
class ResNet(tf.keras.Model):
  def __init__(self):
    super(ResNet, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu') # 28x28x64
        
    self.res1 = ResnetLayer(64, (128, 128), (3, 3)) # 28x28x128
    self.pool1 = tf.keras.layers.MaxPool2D((2, 2)) # 14x14x128
        
    self.res2 = ResnetLayer(128, (256, 256), (3, 3)) # 14x14x256
    self.pool2 = tf.keras.layers.MaxPool2D((2, 2)) # 7x7x256
        
    self.res3 = ResnetLayer(256, (512, 512), (3, 3)) # 7x7x512
        
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
    self.dense2 = tf.keras.layers.Dense(10, activation='softmax')
        
  def call(self, x, training=False, mask=None):
    h = self.conv1(x)
        
    h = self.res1(h, training=training)
    h = self.pool1(h)
    h = self.res2(h, training=training)
    h = self.pool2(h)
    h = self.res3(h, training=training)
        
    h = self.flatten(h)
    h = self.dense1(h)

    return self.dense2(h)

In [9]:
mnist=tf.keras.datasets.mnist

#Load Data from MNIST
(x_train,y_train),(x_test,y_test)=mnist.load_data()

#Data Normalization
x_train,x_test=x_train/255.0,x_test/255.0

#Channel 차원 추가
# x_train=(NUM_SAMPLE,28,28) -> (NUM_SAMPLE,28,28,1)
x_train=x_train[...,tf.newaxis].astype(np.float32)
x_test=x_test[...,tf.newaxis].astype(np.float32)

train_ds=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(16)
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(16)

11493376/11490434 [==============================] - 0s 0us/step


In [10]:
model=ResNet()

loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
optm=tf.keras.optimizers.Adam()

train_loss=tf.keras.metrics.Mean(name='train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss=tf.keras.metrics.Mean(name='test_loss')
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [11]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [12]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optm, train_loss, train_accuracy)
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template='Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),train_accuracy.result()*100,
                          test_loss.result(),test_accuracy.result()*100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.37034785747528076, Accuracy: 94.30000305175781, Test Loss: 0.061627522110939026, Test Accuracy: 98.04999542236328
Epoch 2, Loss: 0.07013050466775894, Accuracy: 97.93333435058594, Test Loss: 0.08395904302597046, Test Accuracy: 97.58999633789062
Epoch 3, Loss: 0.05592624470591545, Accuracy: 98.34333038330078, Test Loss: 0.04979050159454346, Test Accuracy: 98.54000091552734
Epoch 4, Loss: 0.04659480229020119, Accuracy: 98.69833374023438, Test Loss: 0.03775816783308983, Test Accuracy: 98.76000213623047
Epoch 5, Loss: 0.0355028361082077, Accuracy: 98.96333312988281, Test Loss: 0.025634365156292915, Test Accuracy: 99.18000030517578
Epoch 6, Loss: 0.033794812858104706, Accuracy: 99.09832763671875, Test Loss: 0.05007009953260422, Test Accuracy: 98.86000061035156
Epoch 7, Loss: 0.024737047031521797, Accuracy: 99.3116683959961, Test Loss: 0.04208442568778992, Test Accuracy: 98.75
Epoch 8, Loss: 0.02519684098660946, Accuracy: 99.32666778564453, Test Loss: 0.036944374442100525, Te